<a href="https://colab.research.google.com/github/lilianabs/chatbot/blob/main/Chatbot_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
import numpy
import tensorflow
import random
import json
import re

from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords

from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/Intents.json', encoding='utf-8') as file:
    data = json.load(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data['intents'][0]

{'tag': 'Capital',
 'patterns': ['¿Cuál es la capital de Fráncia?',
  '¿Sabes cuál es la capital para España?',
  'En las capitales del mundo, ¿Cuál le pertenece a Inglaterra?',
  'Si te pregunto por la capital de México, ¿Cuál es?',
  'Háblame de cuál es la capital de Canadá'],
 'responses': ['La capital es...']}

In [6]:
labels = []
texts = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [7]:
texts

['¿Cuál es la capital de Fráncia?',
 '¿Sabes cuál es la capital para España?',
 'En las capitales del mundo, ¿Cuál le pertenece a Inglaterra?',
 'Si te pregunto por la capital de México, ¿Cuál es?',
 'Háblame de cuál es la capital de Canadá',
 '¿Cuál es la raíz cuadrada de 12',
 '¿Puedes calcular la raíz cuadrada de 34.67?',
 'Dame la raíz cuadrada de 25',
 'A que no calculas la raíz de 34.56',
 'Calcúlame la raiz de 125.90',
 '¿Qué idioma se habla en Austria?',
 '¿En Austria qué idioma hablan?',
 '¿Sabes cuál es el idioma hablado en austria?',
 '¿En qué idioma me hablan si viajo a Austria?',
 '¿Cuál es el animal más grande del mundo?',
 'Sabes cuál es el animal más grande en el mundo?',
 '¿En todo el mundo cuál es el animál más grande?',
 'Dime cuál animal es más grande en todo el planeta',
 '¿Qué significa ADN?',
 '¿Qué significan las siglas de ADN?',
 '¿Las siglas de AND para qué son utilizadas?',
 '¿Me puedes decir qué significa ADN?',
 '¿Cómo te llamas?',
 '¿Cuál es tu nombre?',
 

In [8]:
labels

['Capital',
 'Raiz_Cuadrada',
 'Idioma_Austria',
 'Animal_mas_grande',
 'ADN',
 'Nombre',
 'Paises_Europa',
 'Critobal_Colon']

Now, we need to generate the output vector so we can use it for training a classification model.

In [9]:
output = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    output.append(labels.index(intent['tag']))

print(output)

In [13]:
train_labels = to_categorical(output, num_classes=len(labels))
print(train_labels)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 

## Data preprocessing

In [17]:
stop_words = stopwords.words('spanish')

In [37]:
def text_preprocessing(texts):
  X = []

  for txt in texts:
    # Remove stop words
    for stop_word in stop_words:
      txt = txt.replace(" " + stop_word + " ", " ")

    # Remove acentos
    txt = txt.replace("á", "a")
    txt = txt.replace("é", "e")
    txt = txt.replace("í", "i")
    txt = txt.replace("ó", "o")
    txt = txt.replace("ú", "u")

    # Remove multiple whitespaces
    txt = re.sub(r'\s+', " ", txt)

    # Lowercase text
    txt = txt.lower()

    # Remove punctuation marks
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(txt)

    X.append(TreebankWordDetokenizer().detokenize(result))

  return X



In [38]:
X = text_preprocessing(texts)

## Prepare data

In [40]:
maxlen_user = 5
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_train = pad_sequences(X_seq, padding='post', maxlen=maxlen_user)

In [41]:
print(X_train)

[[ 1  5 36  0  0]
 [ 6  1  5 37  0]
 [38  8  1 39 40]
 [41  5 42  1 43]
 [23  1  5 44  0]
 [ 1  3 15 45  0]
 [46  3 15 25 47]
 [48  3 15 49  0]
 [16 50  3 25 51]
 [52  3 53 54  0]
 [17  9 55 10  0]
 [ 7 10  9 26  0]
 [ 6  1  9 56 10]
 [ 9 26 22 57 10]
 [ 1 11 12  8  0]
 [ 6  1 11 12  8]
 [ 7  8  1 11 12]
 [58  1 11 12 59]
 [17 27 18  0  0]
 [17 60 28 18  0]
 [61 28 62 63  0]
 [64 24 65 27 18]
 [13 66  0  0  0]
 [ 1 19  0  0  0]
 [13 29 30  0  0]
 [31 14 67  0  0]
 [16 19 68  0  0]
 [69 70 19  0  0]
 [13 29 30  0  0]
 [13 71  0  0  0]
 [20  2 21  0  0]
 [20  2 72 32 33]
 [31 20  2 73 21]
 [34  2 21  0  0]
 [34  2 32 33  0]
 [ 2 74  0  0  0]
 [14 75  4  0  0]
 [ 6 14 76  4  0]
 [16 14 77 35  4]
 [23 35  4  0  0]
 [78 79  4  0  0]]
